In [2]:
!pip install librosa sounddevice tensorflow==2.16.1


In [3]:
import os
import numpy as np
import pandas as pd
import librosa
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

print("TensorFlow:", tf.__version__)


TensorFlow: 2.16.1


In [4]:
AUDIO_DIR = r"C:/Users/Dinupa Devinda/Desktop/ML/Horn/Dataset"   

TRAIN_CSV = "C:/Users/Dinupa Devinda/Desktop/ML/Horn/hornbase_train.csv"
TEST_CSV  = "C:/Users/Dinupa Devinda/Desktop/ML/Horn/hornbase_test.csv"


In [5]:
train_df = pd.read_csv(TRAIN_CSV)
test_df  = pd.read_csv(TEST_CSV)

print(train_df.head())
print("Train samples:", len(train_df))
print("Test samples:", len(test_df))


            file class  class number
0  C_10_A_L_.wav  horn             1
1   C_10_A_S.wav  horn             1
2   C_10_A_V.wav  horn             1
3  C_10_A_V_.wav  horn             1
4   C_10_B_L.wav  horn             1
Train samples: 756
Test samples: 324


In [6]:
import numpy as np
import librosa
from sklearn.model_selection import train_test_split


TARGET_SR  = 44100      
DURATION   = 1.0         
N_SAMPLES  = int(TARGET_SR * DURATION)


N_MELS     = 128        
N_FFT      = 1024
HOP_LENGTH = 512

def wav_to_logmel_raw(path):
    """Load wav and return *unnormalized* log-mel spectrogram."""
    y, sr = librosa.load(path, sr=TARGET_SR, mono=True)

    # Ensure exact length 1 s
    if len(y) < N_SAMPLES:
        y = np.pad(y, (0, N_SAMPLES - len(y)))
    else:
        y = y[:N_SAMPLES]

    mel = librosa.feature.melspectrogram(
        y=y,
        sr=sr,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS,
        power=2.0
    )
    log_mel = librosa.power_to_db(mel, ref=np.max)
    return log_mel.astype("float32")

def load_logmel_dataset(df, audio_dir):
    """Return (X_raw, y) where X_raw is (N, mels, frames) unnormalized."""
    feats = []
    labels = []
    for _, row in df.iterrows():
        fname = row["file"]
        label = row["class number"]   

        full_path = os.path.join(audio_dir, fname)
        if not os.path.exists(full_path):
            raise FileNotFoundError(full_path)

        logmel = wav_to_logmel_raw(full_path)
        feats.append(logmel)
        labels.append(label)

    X_raw = np.stack(feats)                      
    y = np.array(labels, dtype="float32")
    return X_raw, y

print("🔁 Extracting log-mel spectrograms…")
X_all_raw, y_all   = load_logmel_dataset(train_df, AUDIO_DIR)
X_test_raw, y_test = load_logmel_dataset(test_df,  AUDIO_DIR)

print("All train raw:", X_all_raw.shape)
print("Test raw     :", X_test_raw.shape)


X_train_raw, X_val_raw, y_train, y_val = train_test_split(
    X_all_raw, y_all,
    test_size=0.15,
    stratify=y_all,
    random_state=42
)

print("Train raw:", X_train_raw.shape, "Val raw:", X_val_raw.shape)


def random_spec_augment(mel):
    """
    mel: (n_mels, n_frames) log-mel
    returns augmented copy
    """
    v = mel.copy()
    n_mels, n_frames = v.shape

    # Frequency mask
    max_f = max(1, n_mels // 8)   # up to 1/8 of bands
    f = np.random.randint(0, max_f + 1)
    if f > 0:
        f0 = np.random.randint(0, n_mels - f + 1)
        v[f0:f0+f, :] = 0.0

    # Time mask
    max_t = max(1, n_frames // 8)
    t = np.random.randint(0, max_t + 1)
    if t > 0:
        t0 = np.random.randint(0, n_frames - t + 1)
        v[:, t0:t0+t] = 0.0

    return v


X_train_aug_list = []
y_train_aug_list = []

for x, y in zip(X_train_raw, y_train):
    X_train_aug_list.append(x)
    y_train_aug_list.append(y)

    x_aug = random_spec_augment(x)
    X_train_aug_list.append(x_aug)
    y_train_aug_list.append(y)

X_train_aug = np.stack(X_train_aug_list)
y_train_aug = np.array(y_train_aug_list, dtype="float32")

print("Augmented train:", X_train_aug.shape, "labels:", y_train_aug.shape)


mean = X_train_aug.mean()
std  = X_train_aug.std() + 1e-9
print("Global mean/std:", float(mean), float(std))

def normalize_and_expand(X, mean, std):
    Xn = (X - mean) / std
    return Xn[..., np.newaxis].astype("float32") 

X_train = normalize_and_expand(X_train_aug, mean, std)
X_val   = normalize_and_expand(X_val_raw,   mean, std)
X_test  = normalize_and_expand(X_test_raw,  mean, std)

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)


🔁 Extracting log-mel spectrograms…
All train raw: (756, 128, 87)
Test raw     : (324, 128, 87)
Train raw: (642, 128, 87) Val raw: (114, 128, 87)
Augmented train: (1284, 128, 87) labels: (1284,)
Global mean/std: -43.899112701416016 19.589555741356445
Train: (1284, 128, 87, 1) Val: (114, 128, 87, 1) Test: (324, 128, 87, 1)


In [7]:

from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

input_shape = X_train.shape[1:]   
weight_decay = 1e-4

model = models.Sequential([
    layers.Input(shape=input_shape),

    # Block 1
    layers.Conv2D(32, (3, 3), padding='same',
                  kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.2),

    # Block 2
    layers.Conv2D(64, (3, 3), padding='same',
                  kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    # Block 3
    layers.Conv2D(128, (3, 3), padding='same',
                  kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Dropout(0.3),

    # Block 4 (small)
    layers.Conv2D(128, (3, 3), padding='same',
                  kernel_regularizer=regularizers.l2(weight_decay)),
    layers.BatchNormalization(),
    layers.Activation('relu'),
    layers.GlobalAveragePooling2D(),

    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid')   
])

model.summary()

initial_lr = 1e-3
optimizer = tf.keras.optimizers.Adam(learning_rate=initial_lr)

model.compile(
    optimizer=optimizer,
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        tf.keras.metrics.Precision(name='precision'),
        tf.keras.metrics.Recall(name='recall')
    ]
)


BEST_MODEL_PATH = r"C:\Users\Dinupa Devinda\Desktop\ML\Horn\horn_cnn_best.keras"

callbacks = [
    EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True,
        verbose=1
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        min_lr=1e-6,
        verbose=1
    ),
    ModelCheckpoint(
        filepath=BEST_MODEL_PATH,
        monitor='val_loss',
        save_best_only=True,
        verbose=1
    )
]

history = model.fit(
    X_train, y_train_aug,
    validation_data=(X_val, y_val),
    epochs=120,
    batch_size=32,
    callbacks=callbacks,
    shuffle=True
)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 128, 87, 32)         │             320 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 128, 87, 32)         │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation (Activation)              │ (None, 128, 87, 32)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 64, 43, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 64, 43, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 43, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 64, 43, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_1 (Activation)            │ (None, 64, 43, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 32, 21, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 32, 21, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 21, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 32, 21, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_2 (Activation)            │ (None, 32, 21, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 16, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 16, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_3 (Conv2D)                    │ (None, 16, 10, 128)         │         147,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 16, 10, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ activation_3 (Activation)            │ (None, 16, 10, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 128)                 │               0 │
│ (GlobalAveragePooling2D)             │                             │              

 Total params: 258,305 (1009.00 KB)

 Trainable params: 257,601 (1006.25 KB)

 Non-trainable params: 704 (2.75 KB)

Epoch 1/120
40/41 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.6085 - loss: 0.6887 - precision: 0.6036 - recall: 0.5456
Epoch 1: val_loss improved from None to 0.84465, saving model to C:\Users\Dinupa Devinda\Desktop\ML\Horn\horn_cnn_best.keras
41/41 ━━━━━━━━━━━━━━━━━━━━ 8s 131ms/step - accuracy: 0.6472 - loss: 0.6527 - precision: 0.6621 - recall: 0.6012 - val_accuracy: 0.5000 - val_loss: 0.8446 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 2/120
40/41 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step - accuracy: 0.7418 - loss: 0.5569 - precision: 0.7311 - recall: 0.7493
Epoch 2: val_loss did not improve from 0.84465
41/41 ━━━━━━━━━━━━━━━━━━━━ 5s 115ms/step - accuracy: 0.7609 - loss: 0.5316 - precision: 0.7750 - recall: 0.7352 - val_accuracy: 0.5000 - val_loss: 1.0924 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - learning_rate: 0.0010
Epoch 3/120
40/41 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - accuracy: 0.8100 - loss: 0.4542 - precision: 0.8312 - recall: 0

In [8]:
from sklearn.metrics import classification_report, confusion_matrix

test_metrics = model.evaluate(X_test, y_test, verbose=0)
for name, value in zip(model.metrics_names, test_metrics):
    print(f"{name}: {value:.4f}")

# Detailed metrics
y_probs = model.predict(X_test).flatten()
y_pred  = (y_probs >= 0.5).astype("int32")

print("\nClassification report (threshold=0.5):")
print(classification_report(y_test, y_pred, digits=4))

print("Confusion matrix:")
print(confusion_matrix(y_test, y_pred))

# Quick look at first 10 predictions
print("\nFirst 10 test samples:")
for p, prob, label in zip(y_pred[:10], y_probs[:10], y_test[:10]):
    print(f"Prob horn = {prob:.2f} | Pred = {p} | True = {label}")


import os

MODEL_DIR = r"C:\Users\Dinupa Devinda\Desktop\ML\Horn"
os.makedirs(MODEL_DIR, exist_ok=True)

MODEL_PATH = os.path.join(MODEL_DIR, "horn_cnn_model_v2.keras")
model.save(MODEL_PATH)

print("Final model saved to:", MODEL_PATH)
print("Best val-loss model saved to:", BEST_MODEL_PATH)


loss: 0.1158
compile_metrics: 0.9753
11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 44ms/step

Classification report (threshold=0.5):
              precision    recall  f1-score   support

         0.0     0.9695    0.9815    0.9755       162
         1.0     0.9812    0.9691    0.9752       162

    accuracy                         0.9753       324
   macro avg     0.9754    0.9753    0.9753       324
weighted avg     0.9754    0.9753    0.9753       324

Confusion matrix:
[[159   3]
 [  5 157]]

First 10 test samples:
Prob horn = 0.99 | Pred = 1 | True = 1.0
Prob horn = 0.84 | Pred = 1 | True = 1.0
Prob horn = 1.00 | Pred = 1 | True = 1.0
Prob horn = 1.00 | Pred = 1 | True = 1.0
Prob horn = 1.00 | Pred = 1 | True = 1.0
Prob horn = 0.99 | Pred = 1 | True = 1.0
Prob horn = 1.00 | Pred = 1 | True = 1.0
Prob horn = 1.00 | Pred = 1 | True = 1.0
Prob horn = 1.00 | Pred = 1 | True = 1.0
Prob horn = 1.00 | Pred = 1 | True = 1.0
Final model saved to: C:\Users\Dinupa Devinda\Desktop\ML\Horn\horn_cnn_model_v2.k

In [10]:
# after training
np.savez(r"C:\Users\Dinupa Devinda\Desktop\ML\Horn\norm_stats.npz", mean=mean, std=std)


In [ ]:
import time
import numpy as np
import librosa
import sounddevice as sd
import tensorflow as tf

MODEL_PATH = r"C:\Users\Dinupa Devinda\Desktop\ML\Horn\Models\horn_cnn_best.keras"
NORM_PATH  = r"C:\Users\Dinupa Devinda\Desktop\ML\Horn\Models\norm_stats.npz"


TARGET_SR  = 44100
WINDOW_SEC = 1.0
N_SAMPLES  = int(TARGET_SR * WINDOW_SEC)

N_MELS     = 128
N_FFT      = 1024
HOP_LENGTH = 512

STEP_SEC   = 0.25   
BLOCK_SEC  = 0.05   

HORN_THRESHOLD = 0.70

horn_model = tf.keras.models.load_model(MODEL_PATH)
stats = np.load(NORM_PATH)
mean = float(stats["mean"])
std  = float(stats["std"])

print("✅ Loaded model:", MODEL_PATH)
print("✅ Loaded norm stats:", NORM_PATH)
print("mean =", mean, "std =", std)


ring = np.zeros(N_SAMPLES, dtype=np.float32)
write_pos = 0
filled = 0

def add_samples(x: np.ndarray):
    """Add new samples into circular buffer."""
    global ring, write_pos, filled

    x = x.astype(np.float32).flatten()
    n = len(x)

  
    if n >= N_SAMPLES:
        ring[:] = x[-N_SAMPLES:]
        write_pos = 0
        filled = N_SAMPLES
        return

    end = write_pos + n
    if end <= N_SAMPLES:
        ring[write_pos:end] = x
    else:
        part1 = N_SAMPLES - write_pos
        ring[write_pos:] = x[:part1]
        ring[:end - N_SAMPLES] = x[part1:]

    write_pos = (write_pos + n) % N_SAMPLES
    filled = min(N_SAMPLES, filled + n)

def get_last_window():
    """Return latest 1s window in correct time order."""
    if filled < N_SAMPLES:
        return None
    if write_pos == 0:
        return ring.copy()
    return np.concatenate([ring[write_pos:], ring[:write_pos]]).copy()


def audio_to_model_input(y: np.ndarray):
   
    mel = librosa.feature.melspectrogram(
        y=y,
        sr=TARGET_SR,
        n_fft=N_FFT,
        hop_length=HOP_LENGTH,
        n_mels=N_MELS,
        power=2.0
    )
    log_mel = librosa.power_to_db(mel, ref=np.max).astype(np.float32)

    
    log_mel = (log_mel - mean) / (std + 1e-9)

   
    return log_mel[np.newaxis, ..., np.newaxis].astype(np.float32)

def predict_prob(y_1s: np.ndarray) -> float:
    X = audio_to_model_input(y_1s)
   
    prob = float(horn_model(X, training=False).numpy()[0, 0])
    return prob


def audio_callback(indata, frames, time_info, status):
    if status:
    
        pass
   
    add_samples(indata[:, 0])


def listen_continuous():
    print("\n🎤 Continuous horn detection started. Press Ctrl+C to stop.")
    print(f"Window = {WINDOW_SEC}s | Step = {STEP_SEC}s | Threshold = {HORN_THRESHOLD}")

    next_infer = time.time() + WINDOW_SEC  
    ema = None
    alpha = 0.4  

    blocksize = int(TARGET_SR * BLOCK_SEC)

    with sd.InputStream(
        samplerate=TARGET_SR,
        channels=1,
        dtype="float32",
        blocksize=blocksize,
        callback=audio_callback
    ):
        while True:
            now = time.time()
            if now >= next_infer:
                next_infer = now + STEP_SEC

                y = get_last_window()
                if y is None:
                    
                    continue

               
                rms = float(np.sqrt(np.mean(y * y)))
                if rms < 0.005:
                    print("🔇 Silence (rms {:.4f})".format(rms))
                    continue

                prob = predict_prob(y)

               
                ema = prob if ema is None else (alpha * prob + (1 - alpha) * ema)

                msg = f"🔊 Horn prob: {prob:.2f} | smooth: {ema:.2f} | rms: {rms:.4f}"
                if ema >= HORN_THRESHOLD:
                    print(msg, "=> 🚗🔊 HORN DETECTED!")
                else:
                    print(msg)

            time.sleep(0.01)


try:
    listen_continuous()
except KeyboardInterrupt:
    print("\n🛑 Stopped.")


✅ Loaded model: C:\Users\Dinupa Devinda\Desktop\ML\Horn\horn_cnn_best.keras
✅ Loaded norm stats: C:\Users\Dinupa Devinda\Desktop\ML\Horn\norm_stats.npz
mean = -43.899112701416016 std = 19.589555741356445

🎤 Continuous horn detection started. Press Ctrl+C to stop.
Window = 1.0s | Step = 0.25s | Threshold = 0.7
🔇 Silence (rms 0.0000)
🔇 Silence (rms 0.0000)
🔇 Silence (rms 0.0000)
🔇 Silence (rms 0.0000)

🛑 Stopped.
